# ML from Scratch-Multinomial Logistic Regression

When it comes to real-world machine learning, around 70% of the problems are classification-based, where, on the basis of the available set of features, your model tries to predict that out of a given set of categories(discrete possible outcomes), what category does your target variable might belong to. 

**Today, in this article, we are going to have a look at Multinomial Logistic Regression− one of the classic supervised machine learning algorithms capable of doing multi-class classification, i.e., predict an outcome for the target variable when there are more than 2 possible discrete classes of outcomes.
This is a project-based guide, where we will see how to code an MLR model from scratch while understanding the mathematics involved that allows the model to make predictions.**

For the project, we will be working on the famous UCI Cleveland Heart Disease dataset. We will create an ML model from scratch that uses multinomial logistic regression, capable of predicting the severity of heart disease within a patient.

## Importing the Project Dependencies

Let us import the necessary modules.

In [901]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

Now, we will import the dataset. According to the <a href="https://archive.ics.uci.edu/ml/datasets/Heart+Disease">data source</a>, the dataset does not have column names. So we will set the header attribute as None and then we will manually set the column names as per the information available on the source.

In [902]:
col_names = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'num'] #column names

df = pd.read_csv('processed.cleveland.data.csv', header = None)
df.columns = col_names # setting dataframe column names
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0


## Data Preprocessing

Before we actually start working on the model, let us first do preprocessing on our data. The first step is cheching if the dataset has any null values.

In [903]:
for i in df.columns:
    print(df[i].value_counts())

58.0    19
57.0    17
54.0    16
59.0    14
52.0    13
60.0    12
51.0    12
56.0    11
62.0    11
44.0    11
41.0    10
64.0    10
67.0     9
63.0     9
42.0     8
43.0     8
53.0     8
65.0     8
55.0     8
61.0     8
45.0     8
46.0     7
66.0     7
50.0     7
48.0     7
47.0     5
49.0     5
39.0     4
68.0     4
35.0     4
70.0     4
69.0     3
40.0     3
71.0     3
34.0     2
37.0     2
38.0     2
74.0     1
29.0     1
77.0     1
76.0     1
Name: age, dtype: int64
1.0    206
0.0     97
Name: sex, dtype: int64
4.0    144
3.0     86
2.0     50
1.0     23
Name: cp, dtype: int64
120.0    37
130.0    36
140.0    32
110.0    19
150.0    17
138.0    12
128.0    12
125.0    11
160.0    11
112.0     9
132.0     8
118.0     7
124.0     6
135.0     6
108.0     6
152.0     5
134.0     5
145.0     5
100.0     4
170.0     4
122.0     4
136.0     3
105.0     3
115.0     3
142.0     3
126.0     3
180.0     3
178.0     2
94.0      2
148.0     2
146.0     2
102.0     2
144.0     2
172.0     1
117.

As we can see, the columns **ca** and **thal** have 4 and 2 **'?'** values respectively. These are null values that we need to remove.

In [904]:
df.replace({'?': np.nan}, inplace = True) # converting '?' to NaN values
df[['ca', 'thal']] = df[['ca', 'thal']].astype('float64') # Casting columns data-type to floats
df['ca'].replace({np.nan: df['ca'].median()}, inplace = True) # replaces null values of ca column with median value
df['thal'].replace({np.nan: df['thal'].median()}, inplace = True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    float64
 1   sex       303 non-null    float64
 2   cp        303 non-null    float64
 3   trestbps  303 non-null    float64
 4   chol      303 non-null    float64
 5   fbs       303 non-null    float64
 6   restecg   303 non-null    float64
 7   thalach   303 non-null    float64
 8   exang     303 non-null    float64
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    float64
 11  ca        303 non-null    float64
 12  thal      303 non-null    float64
 13  num       303 non-null    int64  
dtypes: float64(13), int64(1)
memory usage: 33.3 KB


Now that we have cleaned our data, let us have a look at the statistical analysis of our dataset.

In [905]:
df.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.438944,0.679868,3.158416,131.689769,246.693069,0.148515,0.990099,149.607261,0.326733,1.039604,1.600660,0.663366,4.722772,0.937294
std,9.038662,0.467299,0.960126,17.599748,51.776918,0.356198,0.994971,22.875003,0.469794,1.161075,0.616226,0.934375,1.938383,1.228536
min,29.000000,0.000000,1.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,1.000000,0.000000,3.000000,0.000000
25%,48.000000,0.000000,3.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,3.000000,0.000000
50%,56.000000,1.000000,3.000000,130.000000,241.000000,0.000000,1.000000,153.000000,0.000000,0.800000,2.000000,0.000000,3.000000,0.000000
75%,61.000000,1.000000,4.000000,140.000000,275.000000,0.000000,2.000000,166.000000,1.000000,1.600000,2.000000,1.000000,7.000000,2.000000
max,77.000000,1.000000,4.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,3.000000,3.000000,7.000000,4.000000


Upon observing the data, we can see that the data needs to be scaled since we have values in the range (1, 1e+2). The main reason we are scaling our data is that since we will be using Stochastic Gradient Descent for optimizing our model parameters, scaling can significantly improve the speed and accuracy of our optimizer.

Here, we will use standard scaling in order to standardize the data.

The first step is to split the dataset into target and feature arrays.

In [906]:
# selecting all the features within our dataset
features = df[['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']] 
features = features.to_numpy() # converts feature set to numpy array
target = df['num'].to_numpy() # converts target column to numpy array
features.shape, len(target) 

((303, 13), 303)

Now, we will define our standard scaling function.

In [907]:
# function for standardizing data
def standardScaler(feature_array):
    """Takes the numpy.ndarray object containing the features and performs standardization on the matrix.
    The function iterates through each column and performs scaling on them individually.
    
    Args-
        feature_array- Numpy array containing training features
    
    Returns-
        None
    """
    
    total_cols = feature_array.shape[1] # total number of columns 
    for i in range(total_cols): # iterating through each column
        feature_col = feature_array[:, i]
        mean = feature_col.mean() # mean stores mean value for the column
        std = feature_col.std() # std stores standard deviation value for the column
        feature_array[:, i] = (feature_array[:, i] - mean) / std # standard scaling of each element of the column

In [908]:
standardScaler(features) # performing standardization on our feature set 

# checking if standardization worked
total_cols = features.shape[1] # total number of columns 
for i in range(total_cols):
    print(features[:, i].std())

0.9999999999999999
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999999999998
1.0
1.0000000000000002
1.0
0.9999999999999999


Now that we have completed the data wrangling process, let us start working on our model.

## Formulating the Model from Scratch

The multinomial regression function consists of two functional layers-
1. Linear prediction function (a.k.a. logit layer)
2. Softmax function (a.k.a. softmax layer)

Let us implement these two function in our model.

First, we need to create randomized weights and biases for our model.

In [909]:
# creating randomized weights for our linear predictor func
weights = np.random.rand(5, 13)
# creating randomized biases for our linear predictor func
biases = np.random.rand(5, 1)

In [910]:
def linearPredict(featureMat, weights, biases):
    """This is the linear predictor function for out MLR model. It calculates the logit scores for each possible outcome.
    
    Args-
        featureMat- A numpy array of features
        weights- A numpy array of weights for our model
        biases- A numpy array of biases for our model
    
    Returns-
        logitScores- Logit scores for each possible outcome of the target variable for each feature set in the feature matrix
    """
    logitScores = np.array([np.empty([5]) for i in range(featureMat.shape[0])]) # creating empty(garbage value) array for each feature set
    
    for i in range(featureMat.shape[0]): # iterating through each feature set
        logitScores[i] = (weights.dot(featureMat[i].reshape(-1,1)) + biases).reshape(-1) # calculates logit score for each feature set then flattens the logit vector 
    
    return logitScores

Now, let us test the function for our feature matrix. The final output should be a 303 x 5 matrix since we have 303 feature sets and 5 target values.

In [911]:
featuresTest = df[['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']]
featuresTest = featuresTest.to_numpy() # converts feature set to numpy array
logitTest = linearPredict(featuresTest, weights, biases)
logitTest.shape

(303, 5)

In [912]:
logitTest[0]

array([311.51259603, 270.623595  , 320.84475682, 519.22643806,
       301.99298163])

As we can see, the function worked just fine. Now to the next step, converting logit scores to probability values. Here's where the softmax function comes into picture. 

What the softmax function does is that it normalizes the logit scores for each possible outcome in a way such that the normalized outputs follow a probabilistic distribution. 
In layman's terms, the softmax function converts logit scores of the possible outcomes of a feature set to probability values.

Now, let us define the softmax function for our model.

In [913]:
def softmaxNormalizer(logitMatrix):
    """Converts logit scores for each possible outcome to probability values.
    
    Args-
        logitMatrix - This is the output of our logitPredict function; consists  logit scores for each feature set
    
    Returns-
        probabilities - Probability value of each outcome for each feature set
    """
    
    probabilities = np.array([np.empty([5]) for i in range(logitMatrix.shape[0])]) # creating empty(garbage value) array for each feature set

    for i in range(logitMatrix.shape[0]):
        exp = np.exp(logitMatrix[i]) # exponentiates each element of the logit array
        sumOfArr = np.sum(exp) # adds up all the values in the exponentiated array
        probabilities[i] = exp/sumOfArr # logit scores to probability values
    return probabilities

Now that we have defined our softmax function as well, let us combine these two functions into a single multinomial function for our model.

In [914]:
def multinomialLogReg(features, weights, biases):
    """Performs logistic regression on a given feature set.
    
    Args- 
        features- Numpy array of features(standardized)
        weights- A numpy array of weights for our model
        biases- A numpy array of biases for our model
    
    Returns-
        probabilities, predictions
        Here,
            probabilities: Probability values for each possible outcome for each feature set in the feature matrix
            predictions: Outcome with max probability for each feature set
    """
    logitScores = linearPredict(features, weights, biases)
    probabilities = softmaxNormalizer(logitScores)
    predictions = np.array([np.argmax(i) for i in probabilities]) #returns the outcome with max probability
    return probabilities, predictions

Now, let us perform logistic regression on our feature set. 

In [915]:
probabilities, predictions = multinomialLogReg(features, weights, biases)
print(probabilities.shape)
print(predictions)

(303, 5)
[3 3 0 2 1 1 2 4 0 2 2 3 1 1 1 4 2 4 1 1 0 3 0 3 0 2 3 3 4 0 3 2 3 2 4 1 0
 3 0 1 3 1 3 1 3 0 1 0 3 2 1 1 1 1 0 0 2 1 2 0 2 4 0 1 2 0 0 3 0 4 3 2 0 3
 1 3 0 3 1 0 0 2 1 3 1 1 1 3 3 3 0 0 3 2 3 1 0 3 1 1 1 1 3 3 2 1 4 3 0 2 3
 2 1 2 3 1 1 2 3 0 1 3 4 0 3 3 3 0 1 2 3 1 1 4 2 3 0 0 4 2 4 3 1 0 0 1 3 1
 1 2 1 2 3 3 0 0 4 3 3 3 1 3 2 0 1 4 1 1 4 1 0 4 4 3 3 0 2 0 1 1 2 3 3 3 3
 3 2 3 3 3 1 0 1 2 0 0 3 4 1 3 1 3 1 3 1 3 0 4 1 4 1 4 4 2 1 3 1 4 2 3 1 1
 1 0 2 1 1 3 0 0 2 3 1 3 3 0 0 1 2 1 1 1 2 3 1 0 1 0 2 1 4 2 0 3 2 2 2 3 3
 1 2 3 3 1 0 4 1 2 1 1 0 3 2 2 3 3 3 2 3 2 0 1 2 1 3 0 2 1 3 2 3 3 4 0 4 1
 2 2 1 2 4 3 1]


Let us now check the accuracy of our model. Since the weights and biases were randomly generated, we can't expect our model to be very accurate at the moment.

In [916]:
def accuracy(predictions, target):
    """Calculates total accuracy for our model.
    
    Args- 
        predictions- Predicted target outcomes as predicted by our MLR function
        target- Actual target values
    
    Returns-
        accuracy- Accuracy percentage of our model
    """
    correctPred = 0
    n = predictions.shape[0]
    for i in range(n):
        if predictions[int(i)] == target[int(i)]:
            correctPred += 1
    accuracy = correctPred/n * 100
    return accuracy

accuracy = accuracy(predictions, target) #calculating accuracy for our model
print(accuracy)

11.881188118811881


As we can see, the initial model accuracy is only about 17%, which is very poor to even consider this model for making any heart disease predictions in real life. 

As a result,  we need to optimize our model parameters in order to improve its accuracy.

## Model Optimization

In this section, we will be working on optimizing our MLR model.

But before we move on towards optimization, let us first split our model into separate training and testing datasets.

In [917]:
def train_test_split(dataframe, test_size = 0.2):
    """Splits dataset into training and testing sets.
    
    Args- 
        dataframe- The dataframe object you want to split
        test_size- Size of test dataset that you want
    
    Returns-
        train_features, train_target, test_features, test_target 
    """
    
    data = dataframe.to_numpy() # converts dataframe to numpy array
    totalRows = data.shape[0] # total rows in the dataset
    testRows = np.round(totalRows * test_size) # total rows in testing dataset
    randRowNum = np.random.randint(0, int(totalRows), int(testRows)) # randomly generated row numbers
    testData = np.array([data[i] for i in randRowNum]) # creates test dataset
    data = np.delete(data, randRowNum, axis = 0) # deletes test data rows from main dataset; making it training dataset
    train_features = data[:, :-1]
    train_target = data[:, -1].astype(int)
    test_features = testData[:, :-1]
    test_target = testData[:, -1].astype(int)
    
    return train_features, train_target, test_features, test_target    

# running train_test_split for our dataset
train_features, train_target, test_features, test_target = train_test_split(df, test_size = 0.17)
standardScaler(train_features) # standard scaling training set 
standardScaler(test_features) # standard scaling testing set
train_features.shape, train_target.shape, test_features.shape, test_target.shape

((255, 13), (255,), (52, 13), (52,))

Now that we have separate training and testing datasets, we will keep the testing dataset aside and only use it for testing purposes. All the training and optimization will be performed on the training dataset.
Let us now go ahead and optimize our model.

First, let us define the loss function. We will be using cross-entropy loss function for our model. For that, we'll be needing one-hot coded target labels.

In [918]:
# creating one-hot-coded target labels for training target labels
oneHotTraining = np.array([np.zeros(5) for i in range(train_target.shape[0])])
for label,i in zip(oneHotTraining,train_target):
    label[int(i)] = 1
    
# creating one-hot-coded target labels for testing target labels
oneHotTest = np.array([np.zeros(5) for i in range(test_target.shape[0])])
for label,i in zip(oneHotTest,test_target):
    label[int(i)] = 1

Now, let us define our loss function.

In [919]:
def crossEntropyLoss(probabilities, target):
    """Calculates cross entropy loss for a set of predictions and actual targets.
    
    Args-
        predictions- Probability predictions, as returned by multinomialLogReg function
        target- Actual target values
    Returns- 
        CELoss- Average cross entropy loss
    """
    n_samples = probabilities.shape[0]
    CELoss = 0
    for sample, i in zip(probabilities, target):
        CELoss += -np.log(sample[i])
    CELoss /= n_samples
    return CELoss       

In [920]:
crossEntropyLoss(probabilities, target)

2.6600187151929244

Now that we have defined our loss function, we will finally define our optimizer algorithm

In [921]:
def stochGradDes(learning_rate, epochs, target, features, weights, biases):
    """Performs stochastic gradient descent optimization on the model.
    
    Args-
        learning_rate- Size of the step the function will take during optimization
        epochs- No. of iterations the function will run for on the model
        target- Numpy array containing actual target values
        features- Numpy array of independent variables
        weights- Numpy array containing weights associated with each feature
        biases- Array containinig model biases
    
    Returns-
        weights, biases, loss_list
        where,
            weights- Latest weight calculated (Numpy array)
            bias- Latest bias calculated (Numpy array)
            loss_list- Array containing list of losses observed after each epoch    
    """
    target = target.astype(int)
    loss_list = np.array([]) #initiating an empty array
    
    for i in range(epochs):
        probabilities, _ = multinomialLogReg(features, weights, biases) # Calculates probabilities for each possible outcome
        
        CELoss = crossEntropyLoss(probabilities, target) # Calculates cross entropy loss for actual target and predictions
        loss_list = np.append(loss_list, CELoss) # Adds the CELoss value for the epoch to loss_list
        
        probabilities[np.arange(features.shape[0]),target] -= 1 # Substract 1 from the scores of the correct outcome
        
        grad_weight = probabilities.T.dot(features) # gradient of loss w.r.t. weights
        grad_biases = np.sum(probabilities, axis = 0).reshape(-1,1) # gradient of loss w.r.t. biases
        
        #updating weights and biases
        weights -= (learning_rate * grad_weight)
        biases -= (learning_rate * grad_biases)
        
    return weights, biases, loss_list

Now, let us run the optimizer on our training set.

In [922]:
updatedWeights, updatedBiases, loss_list = stochGradDes(0.1, 2000, train_target, train_features, weights, biases)

In [923]:
updatedWeights

array([[-2.6943323 ,  1.30404857, -4.80262456, -1.97683343, -0.92090367,
         1.24371714, -2.34408448,  4.54045003, -3.61156067, -3.19822705,
        -4.42638585, -5.10327903, -3.41966719],
       [ 4.47554908, -1.20998259,  2.63898115,  2.48539   ,  3.60802298,
        -0.52963651,  2.90350593, -4.44104526,  4.08056975,  1.35901171,
         2.09361023,  2.20228143,  2.22310557],
       [ 0.20405384, -1.32658598, -0.82459297, -1.03581543,  1.53944925,
         0.97854789, -1.7283219 ,  3.66777448, -1.66084852, -1.49272627,
        -3.42969016, -2.35354879, -2.29110342],
       [-0.98559478, -0.38212998,  2.73590377,  0.09823772, -0.07085267,
         0.20930983,  1.16830304, -1.99383815,  2.1119554 ,  2.32084176,
         3.35499216,  4.08124619,  3.51331167],
       [ 1.83092313,  3.91590813,  2.56635197,  2.248453  , -1.16100602,
        -0.03618447,  1.89695753,  1.91415329,  1.70458123,  2.63553033,
         3.74095162,  4.1814723 ,  1.68629001]])

Finally, we will test our mode with the updated weights and biases. 

**Note- This test will be conducted on the test set.**

In [924]:
testProbabilities, testPredictions = multinomialLogReg(test_features, updatedWeights, updatedBiases)

correctPreds = 0
for i in range(len(testPredictions)):
    if testPredictions[i] == test_target[i]:
        correctPreds += 1
acc = correctPreds / len(testPredictions) * 100
print("Model accuracy on test dataset - {}".format(acc))

Model accuracy on test dataset - 63.46153846153846
